In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns 
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.manifold import TSNE

In [ ]:
os.listdir(os.getcwd())

In [ ]:
file_path2 = '20240612_HLA.csv'
HLA = pd.read_csv(file_path2)

In [ ]:
HLA.columns

In [ ]:
print("Unique values in 'CeD':", HLA['CeD'].unique())

In [ ]:
import pandas as pd
from scipy.stats import chi2_contingency

file_path2 = '20240612_HLA.csv'
HLA = pd.read_csv(file_path2)

# Assuming your DataFrame is already loaded into `HLA`
# Correct the mapping of 'CeD' to be binary
HLA['CeD'] = HLA['CeD'].map({2: 0, 1: 1})

# Create a contingency table with frequencies
contingency_table = pd.crosstab(HLA['DQ_geno_4'], HLA['CeD'])

# Calculate percentages within each CeD group (column-wise percentages)
percentage_table = contingency_table.div(contingency_table.sum(axis=0), axis=1) * 100

# Format the contingency table to "count (percentage%)"
formatted_table = contingency_table.astype(str) + " (" + percentage_table.round(2).astype(str) + "%)"

# Initialize an empty DataFrame for results
results = pd.DataFrame(columns=["Genotype", "Odds Ratio", "P-value"])
for category in contingency_table.index:
    # Create the 2x2 table for each category
    c_table = contingency_table.loc[[category]]
    if c_table.shape[1] < 2:  # Ensures there are two columns
        c_table = pd.concat([c_table, pd.DataFrame([[0, 0]], columns=c_table.columns)], ignore_index=True)
    complement = pd.DataFrame(contingency_table.drop(category).sum()).T
    if complement.shape[1] < 2:
        complement = pd.concat([complement, pd.DataFrame([[0, 0]], columns=complement.columns)], ignore_index=True)
    full_table = pd.concat([c_table, complement], ignore_index=True)

    # Calculate Chi-square test
    chi2, p, dof, expected = chi2_contingency(full_table)

    # Calculate Odds Ratio
    if full_table.iloc[0, 0] != 0 and full_table.iloc[1, 1] != 0:
        OR = (full_table.iloc[0, 0] / full_table.iloc[1, 0]) / (full_table.iloc[0, 1] / full_table.iloc[1, 1])
        OR = f"{OR:.2f}"
    else:
        OR = "Not calculable"

    # Construct new row for results DataFrame
    new_row = pd.DataFrame([[category, OR, f"{p:.2e}"]], columns=["Genotype", "Odds Ratio", "P-value"])
    results = pd.concat([results, new_row], ignore_index=True)

# Print formatted tables
print("Formatted Contingency Table:")
print(formatted_table)
print("\nAnalysis Results:")
print(results)

# Export to CSV
#formatted_table.to_csv("Formatted_Contingency_Table.csv")
#results.to_csv("Analysis_Results.csv")


In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Load and prepare the dataset
file_path2 = '20240612_HLA.csv'
HLA = pd.read_csv(file_path2)
HLA['CeD'] = HLA['CeD'].map({2: 1, 1: 0})

# Encode genotypes and other predictors
significant_genotypes = ['2.2/7.5', '2.5/2.2', '2.5/2.5', '2.5/7.5', '2.5/8.1', '2.5/X']
HLA['DQ_geno_4'] = HLA['DQ_geno_4'].astype(str)
dummies = pd.get_dummies(HLA['DQ_geno_4'])[significant_genotypes]
HLA_encoded = pd.concat([HLA, dummies], axis=1)

In [ ]:
HLA_encoded['genetic_sex'].replace(['male','female'],[0,1],inplace=True)
# Convert and check for multicollinearity
covariates = significant_genotypes + ['PC1', 'PC2', 'PC3', 'genetic_sex', 'age']
HLA_encoded[covariates] = HLA_encoded[covariates].apply(pd.to_numeric, errors='coerce')

HLA_encoded.dropna(subset=covariates,inplace=True)
HLA_encoded

In [ ]:
# Check Variance Inflation Factor (VIF)
vif_data = pd.DataFrame()
vif_data["feature"] = covariates
vif_data


In [ ]:
HLA_encoded

In [ ]:
vif_data["VIF"] = [variance_inflation_factor(HLA_encoded[covariates].values, i) for i in range(len(covariates))]

# Display VIF
print(vif_data)

# Setup and run the logistic regression model with stronger regularization
X = sm.add_constant(HLA_encoded[covariates])  # predictors
y = HLA_encoded['CeD']  # outcome

# Use Elastic Net (combination of L1 and L2 regularization) as an example
model = sm.Logit(y, X)
result = model.fit_regularized(alpha=1.0, L1_wt=0.5)  # Adjust alpha and L1_wt as necessary

# Print the summary
print(result.summary())

In [ ]:
geno_percentages.columns

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['pdf.fonttype']=42
# Assuming HLA is already loaded as a DataFrame
# Group by 'diagnosis domain' and 'DQ_geno', then count the occurrences
geno_counts = HLA.groupby(['diagnosis domain', 'DQ_geno_4']).size().unstack(fill_value=0)

# Normalize the counts to get percentages
geno_percentages = geno_counts.div(geno_counts.sum(axis=1), axis=0) * 100

# Plotting
fig, ax = plt.subplots(figsize=(10, 6))  # Adjust the figure size as needed
geno_percentages.plot(kind='bar', stacked=True, ax=ax, colormap='tab20')

# Adding labels and title
ax.set_ylabel('Percentage')
ax.set_xlabel('Diagnosis Domain')
ax.set_title('Distribution of DQ_geno Across Diagnosis Domains')
ax.legend(['2.5/2.5','2.5/2.2','2.5/7.5','2.5/8.1','2.5/X','2.2/7.5','2.2/2.2','2.2/8.1', '2.2/X', '7.5/7.5', '7.5/8.1', '7.5/X','8.1/8.1',
       '8.1/X', 'X/X'] ,title='HLA-DQ geneotype across different diagnosis', bbox_to_anchor=(1.05, 1), loc='upper left')

# Improve layout to accommodate legend
plt.tight_layout()
plt.savefig('Diag_domain_figure.pdf',format='pdf')


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Load the dataset
file_path2 = '20240612_HLA.csv'
HLA = pd.read_csv(file_path2)

# Define significant genotypes
significant_genotypes = ['2.2/7.5', '2.5/2.2', '2.5/2.5', '2.5/7.5', '2.5/8.1', '2.5/X']

# Group by 'ancestry_pred' and 'DQ_geno', then count the occurrences
geno_counts = HLA.groupby(['ancestry_pred', 'DQ_geno_4']).size().unstack(fill_value=0)

# Normalize the counts to get percentages
geno_percentages = geno_counts.div(geno_counts.sum(axis=1), axis=0) * 100

# Calculate the frequency of 'X/X' for sorting
xx_frequency = geno_counts.get('X/X', 0) / geno_counts.sum(axis=1)

# Add the 'X/X' frequency to the DataFrame for sorting
geno_percentages['XX_Frequency'] = xx_frequency

# Sort the DataFrame based on 'XX_Frequency' ascending (lowest at the top)
geno_percentages_sorted = geno_percentages.sort_values('XX_Frequency', ascending=False)

# Remove the sorting column before plotting
geno_percentages_sorted = geno_percentages_sorted.drop(columns=['XX_Frequency'])

# Define a rainbow color map for significant genotypes and a neutral color for others
color_map = {geno: 'gray' for geno in geno_percentages_sorted.columns}
rainbow_colors = mcolors.TABLEAU_COLORS  # or use mcolors.BASE_COLORS for basic colors
for i, geno in enumerate(significant_genotypes):
    if geno in color_map:
        color_map[geno] = list(rainbow_colors.values())[i % len(rainbow_colors)]
geno_percentages_sorted=geno_percentages_sorted[manual_levels]
# Plotting
fig, ax = plt.subplots(figsize=(12, 8))  # Adjust the figure size as needed for better visibility
geno_percentages_sorted.plot(kind='barh', stacked=True, ax=ax, color=[color_map[col] for col in geno_percentages_sorted.columns])

# Adding labels and title
ax.set_xlabel('Percentage')
ax.set_ylabel('Genetic Ancestry')
ax.set_title('Distribution of DQ_geno Across Genetic Ancestries, Ordered by X/X Frequency')
ax.legend(title='DQ_geno', bbox_to_anchor=(1.05, 1), loc='upper left')

# Improve layout to accommodate legend and longer labels
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Load the dataset
file_path2 = '20240612_HLA2.csv'
HLA = pd.read_csv(file_path2)

# Define significant genotypes
significant_genotypes = ['2.2/7.5', '2.5/2.2', '2.5/2.5', '2.5/7.5', '2.5/8.1', '2.5/X']

# Group by 'ancestry_pred' and 'DQ_geno', then count the occurrences
geno_counts = HLA.groupby(['ancestry_pred', 'DQ_geno_4']).size().unstack(fill_value=0)

# Normalize the counts to get percentages
geno_percentages = geno_counts.div(geno_counts.sum(axis=1), axis=0) * 100

# Calculate the frequency of 'X/X' for sorting
xx_frequency = geno_counts.get('X/X', 0) / geno_counts.sum(axis=1)

# Add the 'X/X' frequency to the DataFrame for sorting
geno_percentages['XX_Frequency'] = xx_frequency

# Sort the DataFrame based on 'XX_Frequency' ascending (lowest at the top)
geno_percentages_sorted = geno_percentages.sort_values('XX_Frequency', ascending=False)

# Remove the sorting column before plotting
geno_percentages_sorted = geno_percentages_sorted.drop(columns=['XX_Frequency'])

# Define a rainbow color map for significant genotypes and a neutral color for others
color_map = {geno: 'gray' for geno in geno_percentages_sorted.columns}
rainbow_colors = mcolors.TABLEAU_COLORS  # or use mcolors.BASE_COLORS for basic colors
for i, geno in enumerate(significant_genotypes):
    if geno in color_map:
        color_map[geno] = list(rainbow_colors.values())[i % len(rainbow_colors)]

# Plotting
fig, ax = plt.subplots(figsize=(12, 8))  # Adjust the figure size as needed for better visibility
geno_percentages_sorted.plot(kind='barh', stacked=True, ax=ax, color=[color_map[col] for col in geno_percentages_sorted.columns])

# Adding labels and title
ax.set_xlabel('Percentage')
ax.set_ylabel('Genetic Ancestry')
ax.set_title('Distribution of DQ_geno Across Genetic Ancestries, Ordered by X/X Frequency')
ax.legend(title='DQ_geno', bbox_to_anchor=(1.05, 1), loc='upper left')

# Improve layout to accommodate legend and longer labels
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Load the dataset
file_path2 = '20240612_HLA2.csv'
HLA = pd.read_csv(file_path2)

# Define significant genotypes
significant_genotypes = ['2.2/7.5', '2.5/2.2', '2.5/2.5', '2.5/7.5', '2.5/8.1', '2.5/X']

# Group by 'ancestry_pred' and 'DQ_geno', then count the occurrences
geno_counts = HLA.groupby(['ancestry_pred', 'DQ_geno_4']).size().unstack(fill_value=0)

# Normalize the counts to get percentages
geno_percentages = geno_counts.div(geno_counts.sum(axis=1), axis=0) * 100

# Calculate the frequency of 'X/X' for sorting
xx_frequency = geno_counts.get('X/X', 0) / geno_counts.sum(axis=1)

# Add the 'X/X' frequency to the DataFrame for sorting
geno_percentages['XX_Frequency'] = xx_frequency

# Sort the DataFrame based on 'XX_Frequency' ascending (lowest at the top)
geno_percentages_sorted = geno_percentages.sort_values('XX_Frequency', ascending=False)

# Remove the sorting column before plotting
geno_percentages_sorted = geno_percentages_sorted.drop(columns=['XX_Frequency'])

# Define a distinct color map for all genotypes
all_colors = plt.cm.get_cmap('tab20', len(geno_percentages_sorted.columns))  # Using a colormap with enough colors
color_map = {geno: all_colors(i) for i, geno in enumerate(geno_percentages_sorted.columns)}

# Assign more distinct colors to significant genotypes using a rainbow colormap
rainbow_colors = plt.cm.get_cmap('nipy_spectral', len(significant_genotypes))
for i, geno in enumerate(significant_genotypes):
    if geno in color_map:
        color_map[geno] = rainbow_colors(i)

# Plotting
fig, ax = plt.subplots(figsize=(12, 8))  # Adjust the figure size as needed for better visibility
geno_percentages_sorted.plot(kind='barh', stacked=True, ax=ax, color=[color_map[col] for col in geno_percentages_sorted.columns])

# Adding labels and title
ax.set_xlabel('Percentage')
ax.set_ylabel('Genetic Ancestry')
ax.set_title('Distribution of DQ_geno Across Genetic Ancestries, Ordered by X/X Frequency')
ax.legend(title='DQ_geno', bbox_to_anchor=(1.05, 1), loc='upper left')

# Improve layout to accommodate legend and longer labels
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Load the dataset
file_path2 = '20240612_HLA2.csv'
HLA = pd.read_csv(file_path2)

# Define significant genotypes
significant_genotypes = ['2.2/7.5', '2.5/2.2', '2.5/2.5', '2.5/7.5', '2.5/8.1', '2.5/X']

# Add a column for CeD status to separate the data easily
HLA['CeD_Status'] = HLA['CeD'].map({1: 'Control', 2: 'Case'})

# Group by 'ancestry_pred', 'CeD_Status' and 'DQ_geno', then count the occurrences
geno_counts = HLA.groupby(['ancestry_pred', 'CeD_Status', 'DQ_geno_4']).size().unstack(fill_value=0)

# Normalize the counts to get percentages
geno_percentages = geno_counts.div(geno_counts.sum(axis=1), axis=0) * 100

# Define a distinct color map for all genotypes
all_colors = plt.get_cmap('tab20')(np.linspace(0, 1, len(geno_counts.columns)))
color_map = {geno: all_colors[i % len(all_colors)] for i, geno in enumerate(geno_counts.columns)}

# Create subplots for each CeD status
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(12, 16), sharex=True)
for i, (status, data) in enumerate(geno_percentages.groupby(level='CeD_Status')):
    ax = axes[i]
    data = data.droplevel('CeD_Status').sort_index(level='ancestry_pred')
    data.plot(kind='barh', stacked=True, ax=ax, color=[color_map[col] for col in data.columns])
    ax.set_title(f'Distribution Across Genetic Ancestries for {status}')
    ax.set_xlabel('Percentage')
    ax.set_ylabel('Ancestry and Genotype')
    ax.legend(title='DQ_geno', bbox_to_anchor=(1.05, 1), loc='upper left')

plt.tight_layout()
plt.show()



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib import rcParams

# Load the dataset
file_path2 = '20240612_HLA2.csv'
HLA = pd.read_csv(file_path2)

# Define significant genotypes
significant_genotypes = ['2.2/7.5', '2.5/2.2', '2.5/2.5', '2.5/7.5', '2.5/8.1', '2.5/X']

# Map CeD values to statuses
HLA['CeD_Status'] = HLA['CeD'].map({1: 'Control', 2: 'Case'})

# Group by 'ancestry_pred', 'CeD_Status' and 'DQ_geno_4', then count the occurrences
geno_counts = HLA.groupby(['ancestry_pred', 'CeD_Status', 'DQ_geno_4']).size().unstack(fill_value=0)

# Normalize the counts to get percentages
geno_percentages = geno_counts.div(geno_counts.sum(axis=1), axis=0) * 100

# Filter to only CeD=2 (Case)
case_data = geno_percentages.xs('Case', level='CeD_Status')

# Define a distinct color map for all genotypes
all_colors = plt.get_cmap('tab20')(np.linspace(0, 1, len(case_data.columns)))
color_map = {geno: all_colors[i % len(all_colors)] for i, geno in enumerate(case_data.columns)}

# Setting the font globally
rcParams['font.family'] = 'Arial'
rcParams['font.size'] = 16

# Plotting
fig, ax = plt.subplots(figsize=(12, 8))
case_data.plot(kind='barh', stacked=True, ax=ax, color=[color_map[col] for col in case_data.columns])
ax.set_title('Distribution Across Genetic Ancestries for Cases (CeD=2)')
ax.set_xlabel('Percentage')
ax.set_ylabel('Ancestry and Genotype')
ax.legend(title='DQ_geno', bbox_to_anchor=(1.05, 1), loc='upper left')

# Save the plot as a PDF file
plt.tight_layout()
plt.savefig('geno_distribution_cases.pdf')

plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib import rcParams

# Load the dataset
file_path2 = '20240612_HLA2.csv'
HLA = pd.read_csv(file_path2)

# Define significant genotypes
significant_genotypes = ['2.2/7.5', '2.5/2.2', '2.5/2.5', '2.5/7.5', '2.5/8.1', '2.5/X']

# Map CeD values to statuses
HLA['CeD_Status'] = HLA['CeD'].map({1: 'Control', 2: 'Case'})

# Group by 'ancestry_pred', 'CeD_Status' and 'DQ_geno_4', then count the occurrences
geno_counts = HLA.groupby(['ancestry_pred', 'CeD_Status', 'DQ_geno_4']).size().unstack(fill_value=0)

# Normalize the counts to get percentages
geno_percentages = geno_counts.div(geno_counts.sum(axis=1), axis=0) * 100

# Filter to only CeD=2 (Case)
case_data = geno_percentages.xs('Case', level='CeD_Status')

# Define a distinct color map for all genotypes
all_colors = plt.get_cmap('tab20')(np.linspace(0, 1, len(case_data.columns)))
color_map = {geno: all_colors[i % len(all_colors)] for i, geno in enumerate(case_data.columns)}

# Set a specific color for X/X genotype
color_map['X/X'] = 'yellow'  # Assign yellow color specifically for the X/X genotype

# Setting the font globally
rcParams['font.family'] = 'Arial'
rcParams['font.size'] = 16

# Plotting
fig, ax = plt.subplots(figsize=(12, 8))
case_data.plot(kind='barh', stacked=True, ax=ax, color=[color_map[col] for col in case_data.columns])
ax.set_title('Distribution Across Genetic Ancestries for Cases (CeD=2)')
ax.set_xlabel('Percentage')
ax.set_ylabel('Ancestry and Genotype')
ax.legend(title='DQ_geno', bbox_to_anchor=(1.05, 1), loc='upper left')

# Save the plot as a PDF file
plt.tight_layout()
plt.savefig('geno_distribution_cases.pdf')

plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib import rcParams

# Load the dataset
file_path2 = '20240612_HLA2.csv'
HLA = pd.read_csv(file_path2)

# Define significant genotypes
significant_genotypes = ['2.2/7.5', '2.5/2.2', '2.5/2.5', '2.5/7.5', '2.5/8.1', '2.5/X']

# Map CeD values to statuses
HLA['CeD_Status'] = HLA['CeD'].map({1: 'Control', 2: 'Case'})

# Update ancestries 'sas', 'mid', 'amr' to 'Other'
HLA['ancestry_pred'] = HLA['ancestry_pred'].replace(['sas', 'mid', 'eas'], 'Other')

# Create dummies for all genotypes
dummies = pd.get_dummies(HLA['DQ_geno_4'])
HLA = pd.concat([HLA, dummies], axis=1)

# Group by 'ancestry_pred', 'CeD_Status' and sum occurrences for each genotype
geno_counts = HLA.groupby(['ancestry_pred', 'CeD_Status'])[dummies.columns].sum()

# Normalize the counts to get percentages
geno_percentages = geno_counts.div(geno_counts.sum(axis=1), axis=0) * 100

# Define a distinct color map for all genotypes
all_colors = plt.get_cmap('tab20')(np.linspace(0, 1, len(dummies.columns)))
color_map = {geno: all_colors[i % len(all_colors)] for i, geno in enumerate(dummies.columns)}

# Set a specific color for X/X genotype
color_map['X/X'] = 'yellow'  # Assign yellow color specifically for the X/X genotype

# Setting the font globally
rcParams['font.family'] = 'Arial'
rcParams['font.size'] = 16

# Plotting
fig, ax = plt.subplots(figsize=(14, 8))
case_data = geno_percentages.xs('Case', level='CeD_Status')
case_data.plot(kind='barh', stacked=True, ax=ax, color=[color_map[col] for col in case_data.columns])
ax.set_title('HLA-DQ Distribution Across Genetic Ancestries')
ax.set_xlabel('Percentage')
ax.set_ylabel('Ancestry and Genotype')
ax.legend(title='HLA-DQ', bbox_to_anchor=(1.05, 1), loc='upper left')

plt.tight_layout()
plt.savefig('geno_distribution_cases.pdf')
plt.show()


In [ ]:
#order the DQ genotypes manually
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib import rcParams
import numpy as np
# Load the dataset
file_path2 = '20241002_HLA.csv'
HLA = pd.read_csv(file_path2)

# Map CeD values to statuses
HLA['CeD_Status'] = HLA['CeD'].map({1: 'Control', 2: 'Case'})

# Update ancestries 'sas', 'mid', 'amr' to 'Other'
HLA['ancestry_pred'] = HLA['ancestry_pred'].replace(['sas', 'mid', 'eas'], 'Other')

# Create dummies for all genotypes
dummies = pd.get_dummies(HLA['Impact label'])
HLA = pd.concat([HLA, dummies], axis=1)

# Group by 'ancestry_pred', 'CeD_Status' and sum occurrences for each genotype
geno_counts = HLA.groupby(['ancestry_pred', 'CeD_Status'])[dummies.columns].sum()

# Normalize the counts to get percentages
geno_percentages = geno_counts.div(geno_counts.sum(axis=1), axis=0) * 100
manual_levels = ['high','moderate','low','none']
geno_percentages=geno_percentages[manual_levels]
# Define a distinct color map for all genotypes
all_colors = plt.get_cmap('tab20')(np.linspace(0, 1, len(dummies.columns)))
color_map = {geno: all_colors[i % len(all_colors)] for i, geno in enumerate(dummies.columns)}

# Set a specific color for X/X genotype
color_map['X/X'] = 'yellow'  # Assign yellow color specifically for the X/X genotype

# Setting the font globally
rcParams['font.size'] = 16
rcParams['pdf.fonttype']=42

# Plotting
fig, ax = plt.subplots(figsize=(14, 8))
case_data = geno_percentages.xs('Case', level='CeD_Status')
print(case_data)
case_data.plot(kind='barh', stacked=True, ax=ax, color=[color_map[col] for col in case_data.columns])
ax.set_title('HLA-DQ Distribution Across Genetic Ancestries')
ax.set_xlabel('Percentage')
ax.set_ylabel('Ancestry and Genotype')
ax.legend(title='HLA-DQ', bbox_to_anchor=(1.05, 1), loc='upper left')

plt.tight_layout()
plt.savefig('geno_distribution_cases_DQ_risk_sorted_20241002.pdf',format='pdf')
#plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib import rcParams
import numpy as np
plt.rcParams['pdf.fonttype']=42
# Load the dataset
file_path2 = '20241002_HLA.csv'
HLA = pd.read_csv(file_path2)

# Map CeD values to statuses
HLA['CeD_Status'] = HLA['CeD'].map({1: 'Control', 2: 'Case'})

# Update ancestries 'sas', 'mid', 'amr' to 'Other'
HLA['ancestry_pred'] = HLA['ancestry_pred'].replace(['sas', 'mid', 'eas'], 'Other')
ttg=pd.read_csv('ttg_iga_measure_count.csv')
seropositive=ttg.loc[ttg['score']==2,'person_id'] #158 total

# Create dummies for all genotypes
dummies = pd.get_dummies(HLA['Impact label'])
HLA = pd.concat([HLA, dummies], axis=1)

# Group by 'ancestry_pred', 'CeD_Status' and sum occurrences for each genotype
geno_counts = HLA.groupby(['ancestry_pred', 'CeD_Status'])[dummies.columns].sum()

# Normalize the counts to get percentages
geno_percentages = geno_counts.div(geno_counts.sum(axis=1), axis=0) * 100
manual_levels = ['high', 'moderate', 'low', 'none']
geno_percentages = geno_percentages[manual_levels]

# Define a distinct color map for all genotypes
all_colors = plt.get_cmap('tab20')(np.linspace(0, 1, len(dummies.columns)))
color_map = {geno: all_colors[i % len(all_colors)] for i, geno in enumerate(dummies.columns)}

# Set a specific color for X/X genotype
color_map['X/X'] = 'yellow'  # Assign yellow color specifically for the X/X genotype

# Setting the font globally
rcParams['font.size'] = 16
rcParams['pdf.fonttype'] = 42

# Plotting
fig, ax = plt.subplots(figsize=(14, 8))
case_data = geno_percentages.xs('Case', level='CeD_Status')
case_data.plot(kind='barh', stacked=True, ax=ax, color=[color_map[col] for col in case_data.columns])

# Annotate bars with percentage values
for i in range(len(case_data)):
    total = 0
    for col in case_data.columns:
        percentage = case_data.iloc[i][col]
        if percentage > 0:  # Only annotate non-zero values
            ax.text(total + percentage / 2, i, f'{percentage:.1f}%', ha='center', va='center', fontsize=10)
        total += percentage

# Set titles and labels
ax.set_title('HLA-DQ Distribution Across Genetic Ancestries')
ax.set_xlabel('Percentage')
ax.set_ylabel('Ancestry and Genotype')
ax.legend(title='HLA-DQ', bbox_to_anchor=(1.05, 1), loc='upper left')

plt.tight_layout()
plt.savefig('geno_distribution_cases_DQ_risk_sorted_20241002.pdf', format='pdf')
#plt.show()
print(geno_counts)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib import rcParams
import numpy as np
plt.rcParams['pdf.fonttype']=42
# Load the dataset
file_path2 = '20241002_HLA.csv'
HLA = pd.read_csv(file_path2)

# Map CeD values to statuses
HLA['CeD_Status'] = HLA['CeD'].map({1: 'Control', 2: 'Case'})

# Update ancestries 'sas', 'mid', 'amr' to 'Other'
HLA['ancestry_pred'] = HLA['ancestry_pred'].replace(['sas', 'mid', 'eas'], 'Other')

#seropositive CeD
ttg=pd.read_csv('ttg_iga_measure_count.csv')
seropositive=ttg.loc[ttg['score']==2,'person_id'] #158 total
HLA_seropositive=HLA[HLA['person_id'].isin(seropositive)]
HLA_seropositive['ancestry_pred']='seropositive CeD'
HLA=pd.concat([HLA[HLA['ancestry_pred']!='Other'],HLA_seropositive])

# Create dummies for all genotypes
manual_levels = ['high', 'moderate', 'low', 'none']
dummies = pd.get_dummies(HLA['Impact label'])
HLA = pd.concat([HLA, dummies], axis=1)
dummies=dummies[manual_levels]
# Group by 'ancestry_pred', 'CeD_Status' and sum occurrences for each genotype
geno_counts = HLA.groupby(['ancestry_pred', 'CeD_Status'])[dummies.columns].sum()

# Normalize the counts to get percentages
geno_percentages = geno_counts.div(geno_counts.sum(axis=1), axis=0) * 100
geno_percentages = geno_percentages[manual_levels]
manual_level_anc = [ 'seropositive CeD', 'amr', 'afr','eur']

# Define a distinct color map for all genotypes
all_colors = plt.get_cmap('coolwarm').reversed()(np.linspace(0, 1, len(dummies.columns)))
color_map = {geno: all_colors[i % len(all_colors)] for i, geno in enumerate(dummies.columns)}

# Set a specific color for X/X genotype
color_map['X/X'] = 'yellow'  # Assign yellow color specifically for the X/X genotype

# Setting the font globally
rcParams['font.size'] = 16
rcParams['pdf.fonttype'] = 42

# Plotting
fig, ax = plt.subplots(figsize=(14, 8))
case_data = geno_percentages.xs('Case', level='CeD_Status').loc[manual_level_anc]
case_data.plot(kind='barh', stacked=True, ax=ax, color=[color_map[col] for col in case_data.columns])


# Set titles and labels
ax.set_title('HLA-DQ Distribution Across Genetic Ancestries')
ax.set_xlabel('Percentage')
ax.set_ylabel('Ancestry and Genotype')
ax.legend(title='HLA-DQ', bbox_to_anchor=(1.05, 1), loc='upper left')

plt.tight_layout()
plt.savefig('geno_distribution_cases_DQ_risk_sorted_20241101.pdf', format='pdf')
plt.show()
print(geno_counts)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib import rcParams
# Load the dataset
file_path2 = '20241002_HLA.csv'
HLA = pd.read_csv(file_path2)

# Define significant genotypes
significant_genotypes = ['2.2/7.5', '2.5/2.2', '2.5/2.5', '2.5/7.5', '2.5/8.1', '2.5/X']

# Map CeD values to statuses
HLA['CeD_Status'] = HLA['CeD'].map({1: 'Control', 2: 'Case'})

# Update ancestries 'sas', 'mid', 'amr' to 'Other'
HLA['ancestry_pred'] = HLA['ancestry_pred'].replace(['sas', 'mid', 'eas'], 'Other')

#manual_level_anc = [ 'seropositive CeD', 'amr', 'afr','eur']

#seropositive CeD
ttg=pd.read_csv('Machine_learning_data_input_20241029.csv')
seropositive=ttg.loc[(ttg['gliadin_igg_score']==2)|(ttg['ttg_igg_score']==1)|(ttg['gliadin_iga_score']==2)|(ttg['ttg_iga_score']==2),'person_id'] #158 total
HLA_seropositive=HLA[HLA['person_id'].isin(seropositive)]
HLA_seropositive['ancestry_pred']='seropositive CeD'
HLA=pd.concat([HLA[HLA['ancestry_pred']!='Other'],HLA_seropositive])

# Create dummies for all genotypes
dummies = pd.get_dummies(HLA['DQ_geno_4'])
HLA = pd.concat([HLA, dummies], axis=1)

# Group by 'ancestry_pred', 'CeD_Status' and sum occurrences for each genotype
geno_counts = HLA.groupby(['ancestry_pred', 'CeD_Status'])[dummies.columns].sum()

# Normalize the counts to get percentages
geno_percentages = geno_counts.div(geno_counts.sum(axis=1), axis=0) * 100
manual_levels = [ '2.5/2.5','2.5/2.2', '2.5/X','2.2/7.5', '2.5/7.5','2.5/8.1','8.1/8.1','2.2/8.1','8.1/X','7.5/8.1','2.2/X','7.5/X','2.2/2.2','7.5/7.5','X/X']
geno_percentages = geno_percentages[manual_levels]

# Define a distinct color map for all genotypes
colors = [ 'firebrick','crimson','deeppink', 'hotpink','palevioletred', 'pink','lightpink','lavenderblush','palegreen','aquamarine','mediumaquamarine','turquoise','lightseagreen','teal','blue']
color_map=dict(zip(manual_levels,colors))

# Calculate total sample size for each ancestry
total_counts_per_ancestry = geno_counts.sum(axis=1).xs('Case', level='CeD_Status')

# Setting the font globally
rcParams['font.family'] = 'Arial'
rcParams['font.size'] = 16
rcParams['pdf.fonttype'] = 42

# Plotting
fig, ax = plt.subplots(figsize=(14, 8))
case_data = geno_percentages.xs('Case', level='CeD_Status')#.loc[manual_level_anc]
case_data.plot(kind='barh', stacked=True, ax=ax, color=[color_map[col] for col in case_data.columns])

# Annotate total sample size for each ancestry
for i, (index, total) in enumerate(total_counts_per_ancestry.items()):
    ax.text(
        1,  # x-coordinate (positioned outside the bar stack)
        i+0.3,  # y-coordinate (same as bar position)
        f'n= {int(total)}',  # text label
        va='center', fontsize=15
    )

ax.set_title('HLA-DQ Distribution Across Genetic Ancestries')
ax.set_xlabel('Percentage')
ax.set_ylabel('Ancestry and Genotype')
ax.legend(title='HLA-DQ', bbox_to_anchor=(1.05, 1), loc='upper left')


    
plt.tight_layout()
plt.savefig('HLA_geno_seropositive_sorted.pdf', format='pdf')
plt.show()


In [ ]:
geno_counts

In [ ]:
high_risk=HLA[HLA['Impact label'].isin(['moderate','high'])&(HLA['CeD_Status']=='Case')]
low_risk=HLA[HLA['Impact label'].isin(['low','none'])&(HLA['CeD_Status']=='Case')]
print(high_risk['age'].mean())
print(low_risk['age'].mean())
print(high_risk['age'].std())
print(low_risk['age'].std())
from scipy.stats import ttest_ind, f_oneway
ttest_ind(high_risk['age'],low_risk['age'])

In [ ]:
case=HLA[HLA['CeD_Status']=='Case']
case.loc[case['ancestry_pred']=='eur','age'].std()
case.loc[case['ancestry_pred'].isin(['afr','amr']),'age'].mean()

ttest_ind(case.loc[case['ancestry_pred']=='eur','age'],case.loc[case['ancestry_pred'].isin(['afr','amr']),'age'])